In [81]:
import os
import pandas as pd
from sklearn import svm
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score
from collections import defaultdict
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier

# Predicting Results based on Game by Game

In [8]:
# What do we do since they changed the seeding on the data with the introduction of the first-four

tournament_result_key = {"0" : "Didn't qualify", "1": "Round of 64", "2": "Round of 32", "4": "Sweet 16", "5": "Elite 8", "7": "Final 4",
                        "8": "Last 2", "9": "Champion"}
tournament_result_after_2010 = {"0" : "Didn't qualify", "1": "First Four", "2": "Round of 64", "3": "Round of 32", "4": "Sweet 16", "5": "Elite 8", "7": "Final 4",
                        "8": "Last 2", "9": "Champion"}

In [9]:
df = pd.read_csv("/Users/eddieloyd/DIS/BigData/final/Big-Data-March-Madness/data/all years normalized.csv")

df = df.rename({'Tourament-Result': 'Tournament-Result'}, axis = 1)
df

/var/folders/q9/42jptm7d151_h70_cp05s8dm0000gn/T/ipykernel_81780/1792856126.py:1: DtypeWarning: Columns (55,56,57,58,65,75,82,83,89,93) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/Users/eddieloyd/DIS/BigData/final/Big-Data-March-Madness/data/all years normalized.csv")


,Season,School,Conference,AP-Pre,AP-High,AP-Final,Tournament-Result,Tournament-Seed,G,W,...,3PAr Normalized,TS% Normalized,TRB% Normalized,AST% Normalized,STL% Normalized,BLK% Normalized,eFG% Normalized,TOV% Normalized,ORB% Normalized,FT/FGA Normalized
0,1992-93,air-force,WAC,30.0,30.0,30.0,0.0,20.0,28,9,...,0.446809,0.306667,0.503311,0.205047,#DIV/0!,0.841155,0.219355,0.403226,#DIV/0!,0.418502
1,1992-93,akron,MAC,30.0,30.0,30.0,0.0,20.0,26,8,...,0.440729,0.420000,0.582781,0.520505,#DIV/0!,0.693141,0.412903,0.483871,#DIV/0!,0.321586
2,1992-93,alabama,SEC,30.0,30.0,30.0,0.0,20.0,29,16,...,0.507599,0.580000,0.622517,0.167192,#DIV/0!,0.815884,0.574194,0.564516,#DIV/0!,0.453744
3,1992-93,alabama-state,SWAC,30.0,30.0,30.0,0.0,20.0,27,14,...,0.215805,0.686667,0.596026,0.271293,#DIV/0!,0.595668,0.606452,0.669355,#DIV/0!,0.603524
4,1992-93,alcorn-state,SWAC,30.0,30.0,30.0,0.0,20.0,27,7,...,0.094225,0.420000,0.370861,0.466877,#DIV/0!,0.754513,0.406452,0.588710,#DIV/0!,0.088106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8556,2017-18,wright-state,Horizon,30.0,30.0,30.0,1.0,14.0,35,25,...,0.332258,0.382022,0.613333,0.547112,0.471264,0.176871,0.363128,0.483146,0.505,0.600000
8557,2017-18,wyoming,MWC,30.0,30.0,30.0,0.0,20.0,33,20,...,0.661290,0.578652,0.300000,0.747720,0.356322,0.401361,0.541899,0.460674,0.02,0.862500
8558,2017-18,xavier,Big East,17.0,3.0,3.0,2.0,1.0,35,29,...,0.364516,0.786517,0.833333,0.686930,0.310345,0.306122,0.743017,0.359551,0.57,1.000000
8559,2017-18,yale,Ivy,30.0,30.0,30.0,0.0,20.0,31,16,...,0.570968,0.567416,0.480000,0.799392,0.45977,0.292517,0.597765,0.505618,0.36,0.381250


In [21]:
# Preprocessing to try and make it so it is game by game instead
# Note data already includes play in games (so no first four)
rounds = pd.read_csv("/Users/eddieloyd/DIS/BigData/final/Big-Data-March-Madness/data/big_dance.csv")

rounds = rounds.dropna(how='all') #remove empty rows


# Obtain only years with practice data

rounds["Start_Year"] = rounds["Season"].str.split('-').str[0].astype(float)
rounds["End_Year"] = rounds["Season"].str.split('-').str[1].astype(float)


In [10]:
# Obtaining list of schools in order to normalize vectors
school_list = []
conference_list = []

for i in range(2,18):
    if i < 9:
        practice_season = "200{}-0{}".format(i, i + 1)
    elif i == 9:
        practice_season = "200{}-{}".format(i, i + 1)
    else:
        practice_season = "20{}-{}".format(i, i + 1)
    # Get a filter year
    practice_year = df.loc[df["Season"].str.contains(practice_season)]
    practice_year = practice_year.fillna(0)
                           
    # Get total list of schools to normalize data
    practice_schools = practice_year["School"].to_numpy()
    
    practice_conf = practice_year["Conference"].to_numpy()
    
    for school in practice_schools:
        if school not in school_list:
            school_list.append(school)
            
    for conf in practice_conf:
    
        if conf not in conference_list:
            if conf == 0:
                conf = "Ind"
            conference_list.append(conf)
            
school_list = list(set(school_list))
school_list.sort()
conference_list = list(set(conference_list))
conference_list.sort()

In [11]:
print(school_list)

['abilene-christian', 'air-force', 'akron', 'alabama', 'alabama-a&m', 'alabama-state', 'albany-(ny)', 'alcorn-state', 'american', 'appalachian-state', 'arizona', 'arizona-state', 'arkansas', 'arkansas-pine-bluff', 'arkansas-state', 'army', 'auburn', 'austin-peay', 'ball-state', 'baylor', 'belmont', 'bethune-cookman', 'binghamton', 'birmingham-southern', 'boise-state', 'boston-college', 'boston-university', 'bowling-green-state', 'bradley', 'brigham-young', 'brown', 'bryant', 'bucknell', 'buffalo', 'butler', 'cal-poly', 'cal-state-bakersfield', 'cal-state-fullerton', 'cal-state-northridge', 'california', 'campbell', 'canisius', 'centenary-(la)', 'central-arkansas', 'central-connecticut-state', 'central-florida', 'central-michigan', 'charleston-southern', 'charlotte', 'chattanooga', 'chicago-state', 'cincinnati', 'clemson', 'cleveland-state', 'coastal-carolina', 'colgate', 'college-of-charleston', 'colorado', 'colorado-state', 'columbia', 'connecticut', 'coppin-state', 'cornell', 'creigh

In [12]:
# Set up school vectors for predicting the winners without names
# And to check winners 
school_indices = dict((c, float(i)) for i, c in enumerate(school_list))
indices_school = dict((float(i), c) for i, c in enumerate(school_list))

In [13]:
# Set up conference vectors for predicting the winners without names
conference_indices = dict((c, float(i)) for i, c in enumerate(conference_list))
indices_conference = dict((float(i), c) for i, c in enumerate(conference_list))
print(conference_indices)

{'A-10': 0.0, 'A-Sun': 1.0, 'AAC': 2.0, 'ACC': 3.0, 'AEC': 4.0, 'Big 12': 5.0, 'Big East': 6.0, 'Big Sky': 7.0, 'Big South': 8.0, 'Big Ten': 9.0, 'Big West': 10.0, 'CAA': 11.0, 'CUSA': 12.0, 'GWC': 13.0, 'Horizon': 14.0, 'Ind': 15.0, 'Ivy': 16.0, 'MAAC': 17.0, 'MAC': 18.0, 'MEAC': 19.0, 'MVC': 20.0, 'MWC': 21.0, 'Mid-Cont': 22.0, 'NEC': 23.0, 'OVC': 24.0, 'Pac-10': 25.0, 'Pac-12': 26.0, 'Patriot': 27.0, 'SEC': 28.0, 'SWAC': 29.0, 'Southern': 30.0, 'Southland': 31.0, 'Summit': 32.0, 'Sun Belt': 33.0, 'WAC': 34.0, 'WCC': 35.0}


In [193]:
# Setting up dictionary to connect teams to their data based on names
connecting_data = defaultdict()
tournament_teams = df


for i in range(2,18):
    if i < 9:
        practice_season = "200{}-0{}".format(i, i + 1)
    elif i == 9:
        practice_season = "200{}-{}".format(i, i + 1)
    else:
        practice_season = "20{}-{}".format(i, i + 1)
        
    year_data = defaultdict()
    year_teams = defaultdict()
    year = str(i+1)
    # Get a filter year
    practice_year = tournament_teams.loc[tournament_teams["Season"].str.contains(practice_season)]
    practice_year = practice_year.fillna(0)
    practice_year = practice_year.reset_index()

    practice_year = practice_year.drop(["Season"], axis =1)
    for ind in practice_year.index:
        
        team = practice_year["School"][ind]

        practice_year["School"][ind] = school_indices[team]
        
        if practice_year["Conference"][ind] == 0:

            practice_year["Conference"][ind] = conference_indices["Southland"]
        else:
            practice_year["Conference"][ind] = conference_indices[practice_year["Conference"][ind]]
        #print(ind, team)
        stats = practice_year.iloc[ind, :]
        # stats = stats.drop("School")
        
        
        drop_columns = [col for col in stats.index if "Normalized" not in col]
        drop_columns.remove("Conference")
        drop_columns.remove("School")
        
        drop_columns.append('Pace Normalized')
        drop_columns.append('ORtg Normalized')
        drop_columns.append('ORB% Normalized')
        drop_columns.append('STL% Normalized')
        drop_columns.append('3P Per Game Normalized')
        drop_columns.append('FG Per Game Normalized')
        drop_columns.append('FGA Per Game Normalized')
        drop_columns.append('MP Normalized')
        drop_columns.append('BLK% Normalized')
        drop_columns.append('TRB% Normalized')
        drop_columns.append('FT/FGA Normalized')
        drop_columns.append('FTA Per Game Normalized')
        drop_columns.append('FT Per Game Normalized')
        drop_columns.append('W-Conf Normalized')
        drop_columns.append('L-Conf Normalized')
        drop_columns.append('W-Away Normalized')
        drop_columns.append('L-Away Normalized')
        drop_columns.append('W-Home Normalized')
        drop_columns.append('L-Home Normalized')
        drop_columns.append('TOV% Normalized')
        drop_columns.append('FTr Normalized')
        drop_columns.append('BLK Per Game Normalized')
        drop_columns.append('PF Per Game Normalized')
        drop_columns.append('3PA Per Game Normalized')
        drop_columns.append('W-L% Normalized')
        drop_columns.append('TRB Per Game Normalized')
        drop_columns.append('eFG% Normalized')
        drop_columns.append('AST% Normalized')
        drop_columns.append('TS% Normalized')
        drop_columns.append('3PAr Normalized')
        drop_columns.append('FT% Normalized')
        drop_columns.append('STL Per Game Normalized')
        drop_columns.append('TOV Per Game Normalized')
        drop_columns.append('ORB Per Game Normalized')
        drop_columns.append('AST Per Game Normalized')
#         drop_columns.append('School')
#         drop_columns.append('Conference')
        drop_columns.append('W Normalized')
        drop_columns.append('L Normalized')
        drop_columns.append('SRS Normalized')
        drop_columns.append('SOS Normalized')
        drop_columns.append('Pts-Tm. Per Game Normalized')
        drop_columns.append('Pts-Opp. Per Game Normalized')
        drop_columns.append('FG% Normalized')
        drop_columns.append('3P% Normalized')
        
        
        for stat in stats.index:
            if isinstance(stats[stat], str):
                try:
                    stats[stat] = float(stats[stat])
                except:
                    print(stat)
                    drop_columns.append(stat)
            
        
        stats = stats.drop(index=drop_columns)
        print(stats.index)
        year_data[team] = stats
        
    connecting_data[practice_season] = year_data

# for year, year_data in connecting_data.items():
#     print(f"Year: {year}")
#     for team, stats in year_data.items():
#         print(f"  Team: {team}")
#         print(f"    Stats: {stats}")

/var/folders/q9/42jptm7d151_h70_cp05s8dm0000gn/T/ipykernel_81780/2157196607.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  practice_year["School"][ind] = school_indices[team]
/var/folders/q9/42jptm7d151_h70_cp05s8dm0000gn/T/ipykernel_81780/2157196607.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  practice_year["Conference"][ind] = conference_indices[practice_year["Conference"][ind]]
/var/folders/q9/42jptm7d151_h70_cp05s8dm0000gn/T/ipykernel_81780/2157196607.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.

Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='o

/var/folders/q9/42jptm7d151_h70_cp05s8dm0000gn/T/ipykernel_81780/2157196607.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  practice_year["School"][ind] = school_indices[team]
/var/folders/q9/42jptm7d151_h70_cp05s8dm0000gn/T/ipykernel_81780/2157196607.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  practice_year["Conference"][ind] = conference_indices[practice_year["Conference"][ind]]
/var/folders/q9/42jptm7d151_h70_cp05s8dm0000gn/T/ipykernel_81780/2157196607.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.

Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='o

/var/folders/q9/42jptm7d151_h70_cp05s8dm0000gn/T/ipykernel_81780/2157196607.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  practice_year["Conference"][ind] = conference_indices[practice_year["Conference"][ind]]
/var/folders/q9/42jptm7d151_h70_cp05s8dm0000gn/T/ipykernel_81780/2157196607.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stats[stat] = float(stats[stat])
/var/folders/q9/42jptm7d151_h70_cp05s8dm0000gn/T/ipykernel_81780/2157196607.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='object')
Index(['School', 'Conference'], dtype='o

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [194]:
predictions_to_round = {"0": 0, "1": 1, "2": 2, "3":3, "4": 4, "5" : 5, "6": 7, "7": 8, "8": 9}
round_to_predict = {"0":0, "1":1, "2":2, "3":3, "4":4, "5":5, "7":6, "8":7, "9": 8}

In [195]:
rounds_2018 = rounds.loc[rounds["Season"] == "2017-2018"]
rounds_2018

,Season,Round,Region Number,Region Name,Seed,Score,School,School.1,Score.1,Seed.1,Unnamed: 10,Unnamed: 11,Unnamed: 12,Start_Year,End_Year
1575,2017-2018,1.0,1.0,South,1.0,54.0,virginia,maryland-baltimore-county,74.0,16.0,NaN,NaN,NaN,2017.0,2018.0
1576,2017-2018,1.0,1.0,South,2.0,68.0,cincinnati,georgia-state,53.0,15.0,NaN,NaN,NaN,2017.0,2018.0
1577,2017-2018,1.0,1.0,South,3.0,73.0,tennessee,wright-state,47.0,14.0,NaN,NaN,NaN,2017.0,2018.0
1578,2017-2018,1.0,1.0,South,4.0,68.0,arizona,buffalo,89.0,13.0,NaN,NaN,NaN,2017.0,2018.0
1579,2017-2018,1.0,1.0,South,5.0,78.0,kentucky,davidson,73.0,12.0,NaN,NaN,NaN,2017.0,2018.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1633,2017-2018,4.0,3.0,East,1.0,71.0,villanova,texas-tech,59.0,3.0,NaN,NaN,NaN,2017.0,2018.0
1634,2017-2018,4.0,4.0,Midwest,1.0,85.0,kansas,duke,81.0,2.0,NaN,NaN,NaN,2017.0,2018.0
1635,2017-2018,5.0,1.0,Final Four,11.0,57.0,loyola-(il),michigan,69.0,3.0,NaN,NaN,NaN,2017.0,2018.0
1636,2017-2018,5.0,2.0,Final Four,1.0,95.0,villanova,kansas,79.0,1.0,NaN,NaN,NaN,2017.0,2018.0


In [196]:
import random
def get_full_bracket(which_round):
    round_results = []
    for ind in which_round.index:
        team1 = which_round["School"][ind]
        score1 = which_round["Score"][ind]
        team2 = which_round["School.1"][ind]
        score2 = which_round["Score.1"][ind]
        
        region_name = which_round["Region Name"][ind]
        path = which_round["Seed"][ind]
        
        r1 = random.randint(0, 100)

        if r1 %2 == 0:
            
            if score1 > score2:
                winner = (1,0)
            else:
                winner = (0,1)
            round_results.append([(team1, team2), winner, (path, 17-path), region_name])
        else:

            if score1 > score2:
                winner = (0,1)
            else:
                winner = (1,0)
            round_results.append([(team2, team1), winner, (17-path, path), region_name])

    return round_results

In [197]:
full_bracket = []
for i in range(1,7):
    round_i = rounds_2018.loc[rounds_2018["Round"] == i]
    full_bracket.append(get_full_bracket(round_i))
print(full_bracket)

[[[('virginia', 'maryland-baltimore-county'), (0, 1), (1.0, 16.0), 'South'], [('georgia-state', 'cincinnati'), (0, 1), (15.0, 2.0), 'South'], [('tennessee', 'wright-state'), (1, 0), (3.0, 14.0), 'South'], [('buffalo', 'arizona'), (1, 0), (13.0, 4.0), 'South'], [('davidson', 'kentucky'), (0, 1), (12.0, 5.0), 'South'], [('miami-(fl)', 'loyola-(il)'), (0, 1), (6.0, 11.0), 'South'], [('texas', 'nevada'), (0, 1), (10.0, 7.0), 'South'], [('creighton', 'kansas-state'), (0, 1), (8.0, 9.0), 'South'], [('texas-southern', 'xavier'), (0, 1), (16.0, 1.0), 'West'], [('north-carolina', 'lipscomb'), (1, 0), (2.0, 15.0), 'West'], [('michigan', 'montana'), (1, 0), (3.0, 14.0), 'West'], [('unc-greensboro', 'gonzaga'), (0, 1), (13.0, 4.0), 'West'], [('ohio-state', 'south-dakota-state'), (1, 0), (5.0, 12.0), 'West'], [('san-diego-state', 'houston'), (0, 1), (11.0, 6.0), 'West'], [('providence', 'texas-a&m'), (0, 1), (10.0, 7.0), 'West'], [('missouri', 'florida-state'), (0, 1), (8.0, 9.0), 'West'], [('radfo

In [198]:
practice_data = []
practice_target = []
for i in range(2,17):
    if i < 9:
        practice_season = "200{}-200{}".format(i, i + 1)
        year = "200{}-0{}".format(i, i + 1)
    elif i == 9:
        practice_season = "200{}-20{}".format(i, i + 1)
        year = "200{}-{}".format(i, i + 1)
    else:
        practice_season = "20{}-20{}".format(i, i + 1)
        year = "20{}-{}".format(i, i + 1)

    year_data = connecting_data[year]
    full_bracket = []

    rounds_year = rounds.loc[rounds["Season"] == practice_season]

    for i in range(1,6):
        round_i = rounds_year.loc[rounds_year["Round"] == i]

        full_bracket.append(get_full_bracket(round_i))

    for team_rounds in full_bracket:
        for game in team_rounds:
            teams = game[0]
            winners = game[1]
            team1 = teams[0]
            team2 = teams[1]

            data1, data2 = year_data[team1], year_data[team2]
            data = np.array(data1 + data2)

            practice_data.append(data)
#                 if practice_season == "2005-2006":
#                     print(year_data[team1].index)


            practice_target.append(winners[0])

X = np.array(practice_data)
Y = np.array(practice_target)
print(X)   

[[268.0 50.0]
 [577.0 29.0]
 [274.0 39.0]
 ...
 [343.0 31.0]
 [377.0 63.0]
 [434.0 29.0]]


In [199]:
def get_bracket_setup(which_round):
    round_results = []
    for ind in which_round.index:
        team1 = which_round["School"][ind]
        score1 = which_round["Score"][ind]
        team2 = which_round["School.1"][ind]
        score2 = which_round["Score.1"][ind]
        
        region_name = which_round["Region Name"][ind]
        path = which_round["Seed"][ind]
        
        r1 = random.randint(0, 100)
        if r1 %2 == 0:
            round_results.append([(team1, team2),  (path, 17-path), region_name])
        else:
            round_results.append([(team2, team1),  (17- path, path), region_name])
    return round_results

In [200]:
bracket = []
for i in range(1,7):
    round_i = rounds_2018.loc[rounds_2018["Round"] == i]
    bracket.append(get_bracket_setup(round_i))
print(bracket)

[[[('virginia', 'maryland-baltimore-county'), (1.0, 16.0), 'South'], [('cincinnati', 'georgia-state'), (2.0, 15.0), 'South'], [('tennessee', 'wright-state'), (3.0, 14.0), 'South'], [('buffalo', 'arizona'), (13.0, 4.0), 'South'], [('kentucky', 'davidson'), (5.0, 12.0), 'South'], [('loyola-(il)', 'miami-(fl)'), (11.0, 6.0), 'South'], [('nevada', 'texas'), (7.0, 10.0), 'South'], [('creighton', 'kansas-state'), (8.0, 9.0), 'South'], [('xavier', 'texas-southern'), (1.0, 16.0), 'West'], [('lipscomb', 'north-carolina'), (15.0, 2.0), 'West'], [('michigan', 'montana'), (3.0, 14.0), 'West'], [('unc-greensboro', 'gonzaga'), (13.0, 4.0), 'West'], [('ohio-state', 'south-dakota-state'), (5.0, 12.0), 'West'], [('houston', 'san-diego-state'), (6.0, 11.0), 'West'], [('providence', 'texas-a&m'), (10.0, 7.0), 'West'], [('florida-state', 'missouri'), (9.0, 8.0), 'West'], [('villanova', 'radford'), (1.0, 16.0), 'East'], [('purdue', 'cal-state-fullerton'), (2.0, 15.0), 'East'], [('stephen-f.-austin', 'texas

In [201]:
from pandas import DataFrame
def simulate_round(single_round, model_name, connecting_data, year):
    # Making data able to predict data and move it to the next round, to simulate tournament
    next_round = []
    eliminated_teams = []
    for game in single_round:
        team1, team2 = game[0]
        seed1, seed2 = game[1]
        region = game[2]
                
        team1_data = connecting_data[year][team1]
        team2_data = connecting_data[year][team2]
        team_data = team1_data+ team2_data
        size  = len(team_data)
        team_data = np.array(team_data)
        
        team_data = team_data.reshape(-1, size)
    
        # For neural network 
        # team_data = scaler.fit_transform(team_data)
        
        pred = model_name.predict(team_data)
        
        outcome = pred[0].round()
        print(pred)
        # Should predict for which team to win

        
        result = []
        if outcome:
            result.append((team1, (region, seed1)))
            eliminated_teams.append(team2)
        
        else:
            result.append((team2, (region, seed2)))
            eliminated_teams.append(team1)
            
        next_round.append(result)
        
    
    return next_round, eliminated_teams

            

In [202]:
def reduce_first_round(simulated_round):
    south =[0] * 4 
    west = [0] * 4 
    east = [0] * 4 
    midwest = [0] * 4 
    for game in simulated_round:
        team , (region, seed) = game[0]
        if seed in (1, 8, 9, 16):
            ind = 0
        elif seed in (2, 7, 10, 15):
            ind = 1
        elif seed in (3, 6, 11, 14):
            ind = 2
        elif seed in (4, 5, 12, 13):
            ind = 3
            
        if region == "South":
            if isinstance(south[ind], list):
                current = south[ind]
                cur_team = current[0]
                current[0] = (cur_team, team)
                cur_seed = current[1]
                current[1] = (cur_seed, seed)
            else:
                south[ind] = [team, seed, region]
                
                
        elif region == "West":
            if isinstance(west[ind], list):
                current = west[ind]
                cur_team = current[0]
                current[0] = (cur_team, team)
                cur_seed = current[1]
                current[1] = (cur_seed, seed)
            else:
                west[ind] = [team, seed, region]
                
        elif region == "East":
            if isinstance(east[ind], list):
                current = east[ind]
                cur_team = current[0]
                current[0] = (cur_team, team)
                cur_seed = current[1]
                current[1] = (cur_seed, seed)
            else:
                east[ind] = [team, seed, region]
        elif region == "Midwest":
            if isinstance(midwest[ind], list):
                current = midwest[ind]
                cur_team = current[0]
                current[0] = (cur_team, team)
                cur_seed = current[1]
                current[1] = (cur_seed, seed)
            else:
                midwest[ind] = [team, seed, region]
    
    final = []
    for i, game in enumerate(south):
        final.append(game)
        final.append(west[i])
        final.append(east[i])
        final.append(midwest[i])
    return final

In [203]:
def reduce_second_round(simulated_round):
    south =[0] * 2 
    west = [0] * 2 
    east = [0] * 2 
    midwest = [0] * 2 
    for game in simulated_round:
        team , (region, seed) = game[0]
        if seed in (1, 8, 9, 16, 4, 5, 12, 13):
            ind = 0
        elif seed in (2, 7, 10, 15,3, 6, 11, 14):
            ind = 1
            
        if region == "South":
            if isinstance(south[ind], list):
                current = south[ind]
                cur_team = current[0]
                current[0] = (cur_team, team)
                cur_seed = current[1]
                current[1] = (cur_seed, seed)
            else:
                south[ind] = [team, seed, region]
                
                
        elif region == "West":
            if isinstance(west[ind], list):
                current = west[ind]
                cur_team = current[0]
                current[0] = (cur_team, team)
                cur_seed = current[1]
                current[1] = (cur_seed, seed)
            else:
                west[ind] = [team, seed, region]
                
        elif region == "East":
            if isinstance(east[ind], list):
                current = east[ind]
                cur_team = current[0]
                current[0] = (cur_team, team)
                cur_seed = current[1]
                current[1] = (cur_seed, seed)
            else:
                east[ind] = [team, seed, region]
        elif region == "Midwest":
            if isinstance(midwest[ind], list):
                current = midwest[ind]
                cur_team = current[0]
                current[0] = (cur_team, team)
                cur_seed = current[1]
                current[1] = (cur_seed, seed)
            else:
                midwest[ind] = [team, seed, region]
    
    final = []
    for i, game in enumerate(south):
        final.append(game)
        final.append(west[i])
        final.append(east[i])
        final.append(midwest[i])
    return final

In [204]:
def reduce_third_round(simulated_round):
    south =[0] 
    west = [0]
    east = [0]
    midwest = [0]
    for game in simulated_round:
        team , (region, seed) = game[0]
        
            
        if region == "South":
            if isinstance(south[0], list):
                current = south[0]
                cur_team = current[0]
                current[0] = (cur_team, team)
                cur_seed = current[1]
                current[1] = (cur_seed, seed)
            else:
                south[0] = [team, seed, region]
                
                
        elif region == "West":
            if isinstance(west[0], list):
                current = west[0]
                cur_team = current[0]
                current[0] = (cur_team, team)
                cur_seed = current[1]
                current[1] = (cur_seed, seed)
            else:
                west[0] = [team, seed, region]
                
        elif region == "East":
            if isinstance(east[0], list):
                current = east[0]
                cur_team = current[0]
                current[0] = (cur_team, team)
                cur_seed = current[1]
                current[1] = (cur_seed, seed)
            else:
                east[0] = [team, seed, region]
        elif region == "Midwest":
            if isinstance(midwest[0], list):
                current = midwest[0]
                cur_team = current[0]
                current[0] = (cur_team, team)
                cur_seed = current[1]
                current[1] = (cur_seed, seed)
            else:
                midwest[0] = [team, seed, region]
    
    final = []
    for i, game in enumerate(south):
        final.append(game)
        final.append(west[i])
        final.append(east[i])
        final.append(midwest[i])
    return final

In [205]:
def reduce_final_four(simulated_round):
    final_four = [0] * 2
    for game in simulated_round:
        team , (region, seed) = game[0]
        
        # South plays east so put them in the same game
        if region == "South":
            if isinstance(final_four[0], list):
                current = final_four[0]
                cur_team = current[0]
                current[0] = (cur_team, team)
                cur_seed = current[1]
                current[1] = (cur_seed, seed)
            else:
                final_four[0] = [team, seed, region]
                
        # West plays midwest so put them in the same game
        elif region == "West":
            if isinstance(final_four[1], list):
                current = final_four[1]
                cur_team = current[0]
                current[0] = (cur_team, team)
                cur_seed = current[1]
                current[1] = (cur_seed, seed)
            else:
                final_four[1] = [team, seed, region]
                
        elif region == "East":
            if isinstance(final_four[0], list):
                current = final_four[0]
                cur_team = current[0]
                current[0] = (cur_team, team)
                cur_seed = current[1]
                current[1] = (cur_seed, seed)
            else:
                final_four[0] = [team, seed, region]
                
        elif region == "Midwest":
            if isinstance(final_four[1], list):
                current = final_four[1]
                cur_team = current[0]
                current[0] = (cur_team, team)
                cur_seed = current[1]
                current[1] = (cur_seed, seed)
            else:
                final_four[1] = [team, seed, region]
    
    return final_four

In [206]:
def reduce_championship(simulated_round):
    championship = [0]
    for game in simulated_round:
        team , (region, seed) = game[0]
        if isinstance(championship[0], list):
            current = championship[0]
            cur_team = current[0]
            current[0] = (cur_team, team)
            cur_seed = current[1]
            current[1] = (cur_seed, seed)
        else:
            championship[0] = [team, seed, region]
                
    return championship

In [207]:
def simulate_tournament(first_round, model_name, connecting_data, year):
    
    # May want to update this so that it returns what teams are out each round as well
    round_eliminated = []
    round_64_games, eliminated = simulate_round(first_round, model_name, connecting_data, year)
    round_64_complete = reduce_first_round(round_64_games)
    round_eliminated.append(eliminated)
    print("Round of 64 winners: ")
    print(round_64_games)
    print("---------------------------------")
    
    round_32_games, eliminated = simulate_round(round_64_complete, model_name, connecting_data, year)
    round_eliminated.append(eliminated)
    round_32_complete = reduce_second_round(round_32_games)
    print("Round of 32 winners: ")
    print(round_32_games)
    print("---------------------------------")
    
    round_16_games, eliminated = simulate_round(round_32_complete, model_name, connecting_data, year)
    round_eliminated.append(eliminated)
    round_16_complete = reduce_third_round(round_16_games)
    print("Sweet 16 winners: ")
    print(round_16_games)
    print("---------------------------------")
    
    elite_8_games, eliminated = simulate_round(round_16_complete, model_name, connecting_data, year)
    round_eliminated.append(eliminated)
    elite_8_complete = reduce_final_four(elite_8_games)
    print("Elite 8 winners: ")
    print(elite_8_games)
    print("---------------------------------")
    
    final_four_complete, eliminated = simulate_round(elite_8_complete, model_name, connecting_data, year)
    round_eliminated.append(eliminated)
    print("Final 4 winners: ")
    print(final_four_complete)
    print("---------------------------------")
    
    championship_round = reduce_championship(final_four_complete)
    championship_complete, eliminated = simulate_round(championship_round, model_name, connecting_data, year)
    round_eliminated.append(eliminated)
    round_eliminated.append([championship_complete[0][0][0]])
    print("Championship results: ")
    print(championship_complete)
    print("---------------------------------")
    return round_eliminated



In [208]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

#normalize, come back to this later
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

model = SVC()
model.fit(X_train, y_train)

# model.fit(X_train_scaled, y_train)

predictions = model.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)

print(predictions)


Accuracy: 0.5053763440860215
Classification Report:
              precision    recall  f1-score   support

           0       0.27      0.25      0.26        32
           1       0.62      0.64      0.63        61

    accuracy                           0.51        93
   macro avg       0.44      0.44      0.44        93
weighted avg       0.50      0.51      0.50        93

[1 0 0 0 0 1 0 1 1 1 0 1 1 0 1 1 0 1 1 1 1 1 1 1 0 1 0 0 1 1 1 1 0 1 1 0 1
 1 1 1 1 0 1 1 1 0 1 1 0 0 1 0 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 0 1 1 0 1
 0 0 0 1 0 1 1 1 0 1 0 1 0 1 1 1 1 1 1]


In [209]:
tree = DecisionTreeClassifier(random_state = 42)
tree.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

In [210]:
rounds_predicted = simulate_tournament(bracket[0], model, connecting_data, "2017-18")

[1]
[0]
[1]
[0]
[0]
[0]
[1]
[0]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[0]
[1]
[0]
[1]
[1]
[1]
[1]
[0]
[1]
[1]
[0]
[0]
[0]
[0]
[1]
[1]
[1]
Round of 64 winners: 
[[('virginia', ('South', 1.0))], [('georgia-state', ('South', 15.0))], [('tennessee', ('South', 3.0))], [('arizona', ('South', 4.0))], [('davidson', ('South', 12.0))], [('miami-(fl)', ('South', 6.0))], [('nevada', ('South', 7.0))], [('kansas-state', ('South', 9.0))], [('xavier', ('West', 1.0))], [('lipscomb', ('West', 15.0))], [('michigan', ('West', 3.0))], [('unc-greensboro', ('West', 13.0))], [('ohio-state', ('West', 5.0))], [('houston', ('West', 6.0))], [('providence', ('West', 10.0))], [('missouri', ('West', 8.0))], [('villanova', ('East', 1.0))], [('cal-state-fullerton', ('East', 15.0))], [('stephen-f.-austin', ('East', 14.0))], [('marshall', ('East', 13.0))], [('west-virginia', ('East', 5.0))], [('st.-bonaventure', ('East', 11.0))], [('arkansas', ('East', 7.0))], [('alabama', ('East', 9.0))], [('pennsylvania', ('Midwest', 16.0))], [

In [211]:
round_finished = {}
for i, a_round in enumerate(rounds_predicted):
    for team in a_round:
        if i > 3:
            round_finished[team] = i + 3
        else:
            round_finished[team] = i + 2
print(round_finished)
    

{'maryland-baltimore-county': 2, 'cincinnati': 2, 'wright-state': 2, 'buffalo': 2, 'kentucky': 2, 'loyola-(il)': 2, 'texas': 2, 'creighton': 2, 'texas-southern': 2, 'north-carolina': 2, 'montana': 2, 'gonzaga': 2, 'south-dakota-state': 2, 'san-diego-state': 2, 'texas-a&m': 2, 'florida-state': 2, 'radford': 2, 'purdue': 2, 'texas-tech': 2, 'wichita-state': 2, 'murray-state': 2, 'florida': 2, 'butler': 2, 'virginia-tech': 2, 'kansas': 2, 'iona': 2, 'bucknell': 2, 'auburn': 2, 'new-mexico-state': 2, 'tcu': 2, 'oklahoma': 2, 'nc-state': 2, 'kansas-state': 3, 'missouri': 3, 'alabama': 3, 'seton-hall': 3, 'georgia-state': 3, 'providence': 3, 'cal-state-fullerton': 3, 'duke': 3, 'miami-(fl)': 3, 'michigan': 3, 'st.-bonaventure': 3, 'syracuse': 3, 'arizona': 3, 'ohio-state': 3, 'west-virginia': 3, 'college-of-charleston': 3, 'davidson': 4, 'unc-greensboro': 4, 'marshall': 4, 'pennsylvania': 4, 'tennessee': 4, 'lipscomb': 4, 'arkansas': 4, 'michigan-state': 4, 'nevada': 5, 'houston': 5, 'stephe

In [212]:
import random
def get_bracket_results(which_round):
    round_results = []
    for ind in which_round.index:
        team1 = which_round["School"][ind]
        score1 = which_round["Score"][ind]
        team2 = which_round["School.1"][ind]
        score2 = which_round["Score.1"][ind]
        
        region_name = which_round["Region Name"][ind]
        path = which_round["Seed"][ind]
        

        if score1 > score2:
            winner = (1,0)
        else:
            winner = (0,1)
        round_results.append([(team1, team2), winner, (path, 17-path), region_name])

    
    return round_results

In [213]:
full_bracket_results = []
for i in range(1,7):
    round_i = rounds_2018.loc[rounds_2018["Round"] == i]
    full_bracket_results.append(get_bracket_results(round_i))
print(full_bracket_results)

[[[('virginia', 'maryland-baltimore-county'), (0, 1), (1.0, 16.0), 'South'], [('cincinnati', 'georgia-state'), (1, 0), (2.0, 15.0), 'South'], [('tennessee', 'wright-state'), (1, 0), (3.0, 14.0), 'South'], [('arizona', 'buffalo'), (0, 1), (4.0, 13.0), 'South'], [('kentucky', 'davidson'), (1, 0), (5.0, 12.0), 'South'], [('miami-(fl)', 'loyola-(il)'), (0, 1), (6.0, 11.0), 'South'], [('nevada', 'texas'), (1, 0), (7.0, 10.0), 'South'], [('creighton', 'kansas-state'), (0, 1), (8.0, 9.0), 'South'], [('xavier', 'texas-southern'), (1, 0), (1.0, 16.0), 'West'], [('north-carolina', 'lipscomb'), (1, 0), (2.0, 15.0), 'West'], [('michigan', 'montana'), (1, 0), (3.0, 14.0), 'West'], [('gonzaga', 'unc-greensboro'), (1, 0), (4.0, 13.0), 'West'], [('ohio-state', 'south-dakota-state'), (1, 0), (5.0, 12.0), 'West'], [('houston', 'san-diego-state'), (1, 0), (6.0, 11.0), 'West'], [('texas-a&m', 'providence'), (1, 0), (7.0, 10.0), 'West'], [('missouri', 'florida-state'), (0, 1), (8.0, 9.0), 'West'], [('villa

In [214]:
rounds_eliminated = []
for full_round in full_bracket_results:
    round_eliminated = []
    for game in full_round:
        if game[1][0] == 0:
            round_eliminated.append(game[0][0])
            if full_round == full_bracket_results[5]:
                rounds_eliminated.append(round_eliminated)
                rounds_eliminated.append([game[0][1]])
        elif game[1][1] == 0:
            round_eliminated.append(game[0][1])
            if full_round == full_bracket_results[5]:
                rounds_eliminated.append(round_eliminated)
                rounds_eliminated.append([game[0][0]])
    if full_round != full_bracket_results[5]:    
        rounds_eliminated.append(round_eliminated)


    
print(rounds_eliminated)


[['virginia', 'georgia-state', 'wright-state', 'arizona', 'davidson', 'miami-(fl)', 'texas', 'creighton', 'texas-southern', 'lipscomb', 'montana', 'unc-greensboro', 'south-dakota-state', 'san-diego-state', 'providence', 'missouri', 'radford', 'cal-state-fullerton', 'stephen-f.-austin', 'wichita-state', 'murray-state', 'st.-bonaventure', 'arkansas', 'virginia-tech', 'pennsylvania', 'iona', 'bucknell', 'college-of-charleston', 'new-mexico-state', 'tcu', 'oklahoma', 'nc-state'], ['maryland-baltimore-county', 'cincinnati', 'tennessee', 'buffalo', 'xavier', 'north-carolina', 'houston', 'ohio-state', 'alabama', 'butler', 'florida', 'marshall', 'seton-hall', 'rhode-island', 'michigan-state', 'auburn'], ['kentucky', 'nevada', 'gonzaga', 'texas-a&m', 'west-virginia', 'purdue', 'clemson', 'syracuse'], ['kansas-state', 'florida-state', 'texas-tech', 'duke'], ['loyola-(il)', 'kansas'], ['michigan'], ['villanova']]


In [215]:
round_finished_actual = {}
for i, a_round in enumerate(rounds_eliminated):
    for team in a_round:
        if i > 3:
            round_finished_actual[team] = i + 3
        else:
            round_finished_actual[team] = i + 2
print(round_finished_actual)

{'virginia': 2, 'georgia-state': 2, 'wright-state': 2, 'arizona': 2, 'davidson': 2, 'miami-(fl)': 2, 'texas': 2, 'creighton': 2, 'texas-southern': 2, 'lipscomb': 2, 'montana': 2, 'unc-greensboro': 2, 'south-dakota-state': 2, 'san-diego-state': 2, 'providence': 2, 'missouri': 2, 'radford': 2, 'cal-state-fullerton': 2, 'stephen-f.-austin': 2, 'wichita-state': 2, 'murray-state': 2, 'st.-bonaventure': 2, 'arkansas': 2, 'virginia-tech': 2, 'pennsylvania': 2, 'iona': 2, 'bucknell': 2, 'college-of-charleston': 2, 'new-mexico-state': 2, 'tcu': 2, 'oklahoma': 2, 'nc-state': 2, 'maryland-baltimore-county': 3, 'cincinnati': 3, 'tennessee': 3, 'buffalo': 3, 'xavier': 3, 'north-carolina': 3, 'houston': 3, 'ohio-state': 3, 'alabama': 3, 'butler': 3, 'florida': 3, 'marshall': 3, 'seton-hall': 3, 'rhode-island': 3, 'michigan-state': 3, 'auburn': 3, 'kentucky': 4, 'nevada': 4, 'gonzaga': 4, 'texas-a&m': 4, 'west-virginia': 4, 'purdue': 4, 'clemson': 4, 'syracuse': 4, 'kansas-state': 5, 'florida-state':

In [216]:
team_names = list(set(round_finished_actual.keys()))
team_names.sort()
predictions = []
actual = []
for team in team_names:
    predictions.append(round_finished[team])
    actual.append(round_finished_actual[team])
    

In [217]:
accuracy = accuracy_score(actual, predictions)
report = classification_report(actual, predictions)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)


Accuracy: 0.3125
Classification Report:
              precision    recall  f1-score   support

           2       0.53      0.53      0.53        32
           3       0.19      0.19      0.19        16
           4       0.00      0.00      0.00         8
           5       0.00      0.00      0.00         4
           7       0.00      0.00      0.00         2
           8       0.00      0.00      0.00         1
           9       0.00      0.00      0.00         1

    accuracy                           0.31        64
   macro avg       0.10      0.10      0.10        64
weighted avg       0.31      0.31      0.31        64

